In [1]:
from IPython.display import display
from copy import copy

import pandas as pd

In [2]:
METHODS = [
    'add', 'sub', 'mul', 'floordiv', 'div', 'truediv', 'mod', 
    'divmod', 'pow', 'lshift', 'rshift', 'and', 'or', 'xor'
]

def overload_method(instance, name):
    """
    
    """
    def __target__(self, *args, **kwargs):
        """
        
        """
        if not 'inplace' in kwargs or not kwargs['inplace']:
            return SkDataFrame(
                getattr(self.df, name)(*args, **kwargs), 
                list(self.steps) + ['%s(*%s)' % (name, args)]
            )
        else:
            getattr(self.df, name)(*args, **kwargs), 
            self.steps.append('%s(*%s)' % (name, args))
            return None
            
    setattr(instance, name, __target__)
    
    
def overload_private_method(instance, name):
    """
    
    """
    def __target__(self, *args, **kwargs):
        """
        
        """
        return SkDataFrame(
            getattr(self.df, name)(*args, **kwargs), 
            list(self.steps) + ['%s(*%s)' % (name, args)]
        )
    setattr(instance, name, __target__)

    
def overload_private_imethod(instance, name):
    """
    
    """
    def __target__(self, *args, **kwargs):
        """
        
        """
        getattr(self.df, name)(*args, **kwargs), 
        self.steps.append('%s(*%s)' % (name, args))
        return self
    setattr(instance, name, __target__)

In [3]:
_df = pd.DataFrame()

PANDAS_DATAFRAME_OBJECTS = [
    attr for attr in dir(_df) 
    if not attr.startswith('_')
]

PANDAS_DATAFRAME_ATTRIBUTES = [
    attr for attr in PANDAS_DATAFRAME_OBJECTS[30:31] 
    if not callable(getattr(_df, attr))
]

PANDAS_DATAFRAME_METHODS = [
    attr for attr in PANDAS_DATAFRAME_OBJECTS 
    if callable(getattr(_df, attr))
]

/home/xmn/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  from ipykernel import kernelapp as app


In [4]:
class SkDataFrame:
    df = None
    steps = []
    
    def __new__(cls, *args, **kwds):
        """
        
        """
        for method in METHODS:
            overload_private_method(cls, '__%s__' % method)
            overload_private_method(cls, '__r%s__' % method)
            overload_private_imethod(cls, '__i%s__' % method)
            
        for method in PANDAS_DATAFRAME_METHODS:
            overload_method(cls, method)
            
        return super(SkDataFrame, cls).__new__(cls)
    
    def __init__(self, *args, **kwargs):
        if args and isinstance(args[0], pd.DataFrame):
            self.df = args[0]
            if len(args) > 1:
                self.steps = list(args[1])
        else:
            self.df = pd.DataFrame(*args, **kwargs)
        
        # magic methods
        self.__repr__ = self.df.__repr__
        self._repr_html_ = self.df._repr_html_
        

In [5]:
skdf = SkDataFrame({'a': [1,5]})
skdf.df, id(skdf)

(   a
 0  1
 1  5, 139766343605664)

In [6]:
skdf.sum()

,0
sum(*()),NaN


In [7]:
skdf.steps

[]

In [8]:
(skdf / 2).df

,a
0,0.5
1,2.5


In [9]:
skdf += 1
print(skdf.steps)
print(skdf.df)

['__iadd__(*(1,))']
   a
0  2
1  6


In [10]:
print((skdf + 1).df)
print((skdf + 1).steps)
skdf.steps

   a
0  3
1  7
['__iadd__(*(1,))', '__add__(*(1,))']


['__iadd__(*(1,))']

In [11]:
skdf_test = 1 + skdf + 1
display(skdf_test.df)
skdf_test.steps

,a
0,4
1,8


['__iadd__(*(1,))', '__radd__(*(1,))', '__add__(*(1,))']

In [12]:
skdf.steps

['__iadd__(*(1,))']

In [13]:
skdf = 1 + skdf + 1
print(skdf.steps)
print(skdf.df)

['__iadd__(*(1,))', '__radd__(*(1,))', '__add__(*(1,))']
   a
0  4
1  8


In [14]:
skdf.df - 1

,a
0,3
1,7
